In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen 

In [2]:
# 협심증, 비만, 고혈압, 뇌졸중, 당뇨병, 천식, 아토피성 피부염, 간염 (급성 바이러스성 간염), 빈혈 (철결핍빈혈), 심근경색증

chronic = ['협심증', '비만', '고혈압', '뇌졸중', '당뇨병', '천식', '아토피', '간염', '빈혈', '심근경색증']
urls = ['http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=117', # 협심증/협심증식 
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=80', # 비만/열량조절
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=29', # 고혈압/고혈압식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=41', # 뇌졸중/뇌졸중식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=46', # 당뇨병/당뇨병식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=104', # 천식/천식식딘
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=77', # 아토피/아토피성 피부염식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=59', # 간염/바이러스성 간염식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=63', # 빈혈/빈혈
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=75'] # 심근경색증/심근경색식

In [3]:
def bs_web(url):
  # urlopen 함수를 통해 web 변수를 생성
  web = urlopen(url)   # urlopen(url).read().decode('utf-8')

  # BeautifulSoup으로 web 페이지상의 HTML 구조를 파싱
  web_page = BeautifulSoup(web, 'html.parser')

  return web_page

In [4]:
pages = {}

for i in range(len(urls)):
  # get url
  url = urls[i]

  # key == disease
  # value == diet description
  # pages[chronic[i]] = bs_web(url).find('div', {'class': 'contDescription'}).get_text()
  pages[chronic[i]] = bs_web(url).find('div', {'class': 'contDescription'})

In [5]:
# 질환들의 정보 구간별로 나누고 dict형태로 저장하기
def make_dict(disease):
  disease = str(disease).split('<dt class="dep2Title">')
  disease = disease[1:len(disease)-1] # 처음과 마지막 (불필요한 정보) 제거
  d_dict = dict(x.split("</dt>\n<dd><p>") for x in disease) # dict로 만들기
  return d_dict

# dictionary의 value에 있는 불필요한 html 태그 제거하기
def clean(d_dict):
  for k,v in d_dict.items():
    d_dict[k] = BeautifulSoup(d_dict[k], 'html.parser').get_text().strip() # 불필요한 html 태그 제거
    # parser의 get_text()로 제거되지 않는 태그들 제거하기 ["\xa0", "\n", "\r"]
    d_dict[k] = d_dict[k].replace("\r\n", " ")
    d_dict[k] = d_dict[k].replace("\xa0", "")
  return d_dict

In [6]:
# 모든 질환을 포함한 dictionary
disease_dict = {}
disease_dict['angina_pectoris'] = pages['협심증']
disease_dict['obesity'] = pages['비만']
disease_dict['primary_hypertension'] = pages['고혈압']
disease_dict['cerebrovascular_disease'] = pages['뇌졸중']
disease_dict['diabetes'] = pages['당뇨병']
disease_dict['asthma'] = pages['천식']
disease_dict['atopic_dermatitis'] = pages['아토피']
disease_dict['hepatitis'] = pages['간염']
disease_dict['anemia'] = pages['빈혈']
disease_dict['myocardial_infarction'] = pages['심근경색증']

In [7]:
for k,v in disease_dict.items():
  disease_dict[k] = clean(make_dict(disease_dict[k]))

In [8]:
# 빈혈 예시 출
disease_dict['anemia']

{'권장 식품': '색이 짙은 살코기, 생선, 계란, 우유 및 유제품, 색이 짙은 채소, 과일',
 '그 외 주의사항': '조혈모세포이식 직후에는 멸균식(sterile diet)을 하도록 합니다. 1. 동물성 식품은 반드시 익혀서 섭취합니다. 2. 야채는 익혀서, 과일은 세정 후 껍질을 벗겨서 섭취합니다. 3. 시판 제품 중 살균, 멸균 조리된 제품을 이용합니다. 4. 조리된 식품은 2시간 이내에 냉장보관하며, 24시간내 먹도록 합니다. 5. 조리 전, 식사 전에 손을 깨끗이 씻도록 합니다. 6. 가공식품은 유통기한을 확인하도록 합니다.',
 '식사요법의 실제': '1. 규칙적이고 다양한 식품섭취를 합니다. 2. 스테로이드 사용 시 과다한 염분 섭취는 삼가합니다. 3. 구강내 점막염이 발생한 경우 지나치게 차거나 뜨거운 음식, 향이 강한 식품, 신 음식, 거친 음식의 섭취는 삼가하도록 합니다. 4. 매끼 살코기, 생선, 계란 등의 동물성 단백질을 규칙적으로 섭취합니다. 5. 철분 흡수를 돕는 비타민C의 보충을 위해 과일이나 채소류를 규칙적으로 섭취합니다. 6. 철의 흡수를 방해하는 커피, 홍차, 녹차 등은 식사 전후를 피해서 마시도록 합니다.',
 '식사요법의 필요성': '철 결핍성 빈혈은 가장 흔하게 나타나는 빈혈로, 적혈구를 만드는 주요한 영양소인 철의 부족으로 헤모글로빈 합성이 저하되어 발생합니다. 성인의 하루 철 섭취권장량은 남자 10㎎, 여자 14㎎ 로 권고되고 있습니다. 무엇보다 거의 모든 영양소가 혈액을 만드는데 필요하므로 골고루 균형있게 섭취하는 것이 매우 중요합니다.',
 '주의 식품': '카페인 음료, 녹즙, 엑기스, 날 음식'}

In [9]:
# 권장식품 dictionary 만들기
rec_food = {}
rec_food = dict(disease_dict)
for disease in rec_food.keys():
  rec_food[disease] = disease_dict[disease]['권장 식품']

In [10]:
rec_food

{'anemia': '색이 짙은 살코기, 생선, 계란, 우유 및 유제품, 색이 짙은 채소, 과일',
 'angina_pectoris': '신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 등 균형잡힌 식사',
 'asthma': '살코기, 생선, 두부, 신선한 채소 및 과일',
 'atopic_dermatitis': '식품 알레르기와 같이 있는 경우 알레르기 원인 식품을 제외한 모든 음식, 특히 단백질과 아연이 풍부한 육류를 충분히 섭취하는 것이 좋습니다',
 'cerebrovascular_disease': "1. 물, 주스, 우유 등 액상 형태의 대부분의 식품은 '점도증진제(음식의 점도 조절용)'를 섞어서 섭취할 수 있습니다. 2. 삼킴 장애가 심하고 씹지 못 할 때는 밥보다는 죽을 쑤어 블렌더로 곱게 갈아 꿀농도만큼 걸쭉하게 드시길 권장합니다. 반찬(고기, 생선살 나물 등)도 익힌 후 죽을 함께 넣고 블렌더로 곱게 갈아 걸쭉하게 드실 수 있습니다. 3. 우유 대신 요거트를 권장합니다.",
 'diabetes': '채소류, 열량이 적은 식품(곤약, 해조류, 버섯류, 우뭇가사리, 등)',
 'hepatitis': '밥, 떡, 감자, 고구마, 과일, 야채, 단백질 식품( 살코기, 생선, 계란, 두부, 콩 등 )',
 'myocardial_infarction': '신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 등 균형잡힌 식사',
 'obesity': '채소류, 해조류, 곤약, 무가당 차음료(녹차 등), 물(보리차, 생수, 탄산수 등)',
 'primary_hypertension': '신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유 등 균형잡힌 식사'}

In [1]:
import pandas as pd

In [25]:
import pandas as pd

df_recipe = pd.read_csv('food_recipe_DB.csv',index_col='Unnamed: 0')
pd.options.display.max_colwidth

df_recipe.loc[df_recipe['재료'].str.contains(__식자재명__,na=True)]

# df_recipe에 '점수'열 추가
# 질환별 좋은 식재료 리스트. for문 돌려, 해당 식재료를 포함하는 레시피의 점수열 값에 1점씩 더하는 방식.비권장 식재료가 포함될 경우 1점씩 차감
# 루프가 끝나면 가장 많은 점수를 얻은 레시피 top5를 보여주자 


# 아이템기반 유사도 추천 시스템의 경우에는, 유저가 레시피A 선택 -> 선택된 레시피A와 다른 레시피간의 코사인 유사도 도출 -> top5보여주는 방식...?


,음식명,재료,이미지,만드는 방법
85,동태 백김치 말이,"주재료\n백김치 20g(1장), 깻잎 2g(2장), 동태포 60g(2장), 부추 10g,\n샐러리 5g(6cm), 쪽파 10g, 황파프리카 10g(1/10개), 후춧가루 0.1g\n소스\n비트 0.5g, 오미자원액 5g(1작은술), 백김치 국물 5g(1작은술)",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00156_2.png,"1. 비트는 물에 담가 비트 물을 만든다.a2. 동태포는 후춧가루로 밑간을 한다.b3. 찜통에 동태포를 넣고 8분간 찐다.4. 샐러리, 황파프리카, 부추는 6cm로 채 썬다.c5. 끓는 물에 쪽파를 넣어 데친다.6. 백김치에 깻잎, 동태포, 파프리카, 샐러리, 부추를 넣어 만다.7. 데친 쪽파로 백김치를 묶어 고정시킨다.8. 3cm로 잘라 접시에 올린다.9. 비트 물에 백김치국물과 오미자원액을 넣어 소스를 만든 후 접시에 끼얹\n는다."
133,토마토소고기찜,"소고기우둔살 200g, 샐러리 40g, 당근 30g, 양파 50g\n화이트와인 25g, 홀토마토 100g, 버터 20g, 레몬 25g, 흰후추 1g, 파슬리가루 2g, 함초소금 1g",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00392_2.png,"1. 당근, 양파, 샐러리는 사방 2cm 정도로\n잘라준다.2. 레몬은 껍질을 아주 곱게 채 썰어준다.3. 소고기는 사방 3cm 정도로 자르고 흰 후추를\n뿌려준다.4. 홀토마토는 먹기 좋게 부수어 준 후 볶아준다.5. 팬에 버터를 넣고 소고기부터 볶기 시작하다\n채소를 넣고 볶아준다. 이 때 두꺼운 채소부터\n볶아주는 것이 고기랑 익는 시간이 같아진다.6. 마지막에 다져 놓은 홀토마토를 넣고 함초소금\n으로 간을 한 후 레몬껍질을 올려서 마무리\n한다."
141,누룽닭,"닭가슴살(120g), 생강즙(5g), 정종(10g), 시금치(20g)\n당근(20g), 양파(20g), 샐러리(20g), 파프리카(20g)\n느타리(20g), 주꾸미(30g), 새우(20g), 청경채(20g)\n누룽지(50g), 튀김기름(200g), 육수(200g), 저염간장(10g)\n녹말가루(10g), 마늘(10g), 소금(0.2g), 후춧가루(0.1g)",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00500_2.png,"1. 닭은 포를 떠서 생강즙과 정종을 섞어\n발라 놓는다.2. 시금치는 살짝 데쳐 썰고, 당근, 양파,\n샐러리, 파프리카는 약 6Cm 길이로\n채를 썰어 절반은 닭고기말이에 사용\n하고, 절반은 곁들임채소로 사용한다.3. 느타리는 끓는 물에 소금을 넣고 데쳐\n손으로 찢어놓고, 포를 뜬 닭가슴살에\n느타리와 썰어 놓은 채소를 넣어 말아\n주고, 육수를 끓여서 말아놓은 닭을\n넣고 익혀낸다.4. 주꾸미와 새우는 끓는 물에 살짝 데치고,\n청경채는 반으로 썰어 살짝 데쳐 준비\n한다.5. 튀김기름 온도가 180℃ 정도 되면\n누룽지를 넣고 튀겨 기름을 밭쳐 놓는다.6. 닭을 넣은 육수(③)에 간장과 녹말,\n마늘, 소금, 후춧가루를 넣고 끓이다가,\n곁들임채소로 남겨 놓은 채소(②)와\n데쳐 놓은 해물 및 청경채를 넣고 한 번 더\n끓인 뒤 튀겨진 누룽지를 넣어 완성한다."
224,해물아란치니,"오징어(100g), 새우(3마리), 토마토(50g), 양파(30g),\n마늘(20g), 샐러리(20g), 밥(100g), 달걀(1개), 어린잎(10g),\n소금(0.2g), 레몬(1/4개), 빵가루(20g)",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00582_2.png,"1. 새우는 껍질을 벗겨 다진다.2. 오징어는 썰어서 다진다.3. 양파, 샐러리, 마늘은 다져 팬에 볶아\n식힌다.4. 토마토는 껍질을 벗겨 다진다.5. 다진 토마토를 팬에 볶다가 다진\n새우와 오징어, 밥을 넣어 볶고, 3번을\n넣고 볶아 동그랗게 모양을 만든다6. 동그란 밥에 레몬을 섞은 달걀노른자를\n묻히고 빵가루 옷을 입혀 팬에 기름을\n둘러 굴려가며 익힌 후 접시에 담고\n어린잎을 올린다."
245,함초떡갈비,"갈은 소고기 150g, 갈은 돼지고기 150g, 당근 25g, 새송이버섯 20g\n함초 30g, 양파 25g, 샐러리 10g, 대파 10g,\n다진마늘 10g, 참기름 15g, 통깨 5g, 함초소금 1g\n소스 : 발사믹식초 30g, 설탕 15g, 흰후추 1g",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00326_2.png,"1. 함초는 송송 다져준다.2. 당근, 새송이, 샐러리, 양파를 송송 다져준다.3. 볼에 소고기, 돼지고기, 다져 놓은 채소와\n다진마늘, 다진대파, 참기름을 넣고 반죽해준다.4. 여기에 함초 다진 것을 넣고 다시 반죽해 준다.5. 떡갈비 모양으로 만들어준 후 팬에 노릇하게\n구워준다.6. 소스를 만들어 구워진 떡갈비를 졸여준다."
304,채소어묵,"생선살(100g), 맛술(10g), 생강(10g), 소금(0.2g)\n밀가루(20g), 양파(20g),당근(20g), 샐러리(10g), 마늘(10g)\n옥수수(20g), 어린이치즈(20g), 튀김기름(400g)",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00473_2.png,"1. 생선살은 가시를 발라내고 곱게 으깨어\n놓는다.2. 으깬 생선살에 맛술 과 생강즙, 소금을\n넣어 골고루 치댄다.3. 잘 치댄 생선살에 밀가루를 넣어 다시\n치대어 준다.4. 양파와 당근, 샐러리, 마늘은 곱게\n다지고, 옥수수는 알맹이를 준비한다.5. 생선살에 다진 채소와 어린이 치즈를\n다져 섞는다.6. 채소를 다져 섞은 생선살을 동글동글\n하게 완자를 빚어 튀김기름에 약 150℃\n온도에서 은근히 5분 정도 튀겨 완성\n한다."
320,카레닭 룰라이드,"닭고기(120g), 카레가루(20g), 당근(20g), 양파(20g),\n샐러리(1/2개), 아스파라거스(3개), 토마토(50g), 감자(50g),\n생크림(30g), 소금(0.1g), 후춧가루(0.01g)",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00608_2.png,"1. 닭살에 소금, 후추로 밑간을 한다.2. 밑간한 닭살에 카레가루를 골고루\n묻힌다.3. 양파, 당근, 토마토, 샐러리는 주사위\n모양으로 자른다4. 냄비에 양파, 당근, 토마토, 샐러리를\n볶다가 카레가루를 넣고 끓여\n카레소스를 만든다.5. 팬에 기름을 두르고 닭살을 익힌다.6. 그릇에 익힌 닭살을 담고 카레소스를\n올린다."
324,해산물 훈제연어롤,"연어(훈제연어, 120g), 주꾸미(100g), 소라(50g), 완두콩(20g),\n샐러리(30g), 파프리카(1/2개), 오이(20g), 당근(20g),\n양파(20g), 고추장(20g), 식초(20g), 설탕(20g), 소금(0.3g)",http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00612_2.png,"1. 훈제 연어는 넓게 썬다.2. 당근, 오이, 파프리카, 양파, 샐러리는\n길게 썬다.3. 주꾸미는 소금에 주물러 씻는다.4. 손질한 주꾸미, 소라는 끓는 물에\n데치고 썰어준다.5. 고추장에 식초, 설탕을 넣고\n초고추장소스를 만든다.6. 훈제 연어에 야채와 주꾸미, 소라를\n넣고 돌돌 말아 접시에 담고\n초고추장소스를 올리고 완두콩으로\n장식한다."
659,호박잎다슬기된장국,NaN,http://www.foodsafetykorea.go.kr/uploadimg/20141118/20141118102116_1416273676261.jpg,"1. 물 1컵 반에 국멸치와 건다시마를 넣고 멸치다시마국물 1컵을 낸다.2. 호박잎은 줄기를 꺾으며 잡아당겨 실 같은 껍질을 벗기고 씻는다.3. 멸치다시마국물 1컵에 된장을 풀고 씻은 다슬기살을 함께 넣어 중불에서 5분정도 국물을 우려낸다.4. 국물낸 다슬기는 건져 달걀물과 함께 섞는다.5. 부추는 1cm 가량의 크기로 썰고 대파는 어